In [21]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

In [22]:
path = 
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [23]:
path = 
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [24]:
path = 
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

In [25]:
path = 
filename = 'Pivot_NO_Privat_ohneApple_Langzeitkunde.csv'

df_NO_werte = pd.read_csv(os.path.join(path, filename))

In [26]:
path = 
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [27]:
## Alle DataFrame Werte von Float zu int
df_trans_werte = df_trans_werte.astype(int)
df_LY = df_LY.astype(int)
df_HY = df_HY.astype(int)
df_NO_werte = df_NO_werte.astype(int)
df_LO = df_LO.astype(int)

# Transwerte-1 definiert für spätere Berecnungen in der Copula
#df_trans_werte_minusEins = df_trans_werte.loc[:, df_NO_werte.columns != 'iid'] -1
#df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

In [28]:
# Entfernen der Zeilen aus df_HY, die nicht in df_trans_werte und df_LO enthalten sind
df_HY = df_HY[df_HY['iid'].isin(df_trans_werte['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LO['iid'])]

# Entfernen der Zeilen aus df_trans_werte, die nicht in df_HY, df_NO_werte und df_LY enthalten sind
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_NO_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_LY['iid'])]

# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LO = df_LO[df_LO['iid'].isin(df_HY['iid'])]

# Entfernen der Zeilen aus df_NO_werte, die nicht in df_trans_werte enthalten sind
df_NO_werte = df_NO_werte[df_NO_werte['iid'].isin(df_trans_werte['iid'])]

# Entfernen der Zeilen aus df_LY, die nicht in df_trans_werte enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_trans_werte['iid'])]

### Alle DataFrames Indizes resetten, damit man sie gemeinsam identifiziere kann
df_HY = df_HY.reset_index(drop=True)
df_trans_werte = df_trans_werte.reset_index(drop=True)
df_LO = df_LO.reset_index(drop=True)
df_NO_werte = df_NO_werte.reset_index(drop=True)
df_LY = df_LY.reset_index(drop=True)

# NO-1 definiert für spätere Berechnung inder Copula
df_NO_minusEins = df_NO_werte.copy()
df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] = df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] -1
df_NO_minusEins = df_NO_minusEins.astype(int)
# transwerte -1 definiert für spätere Berechnung inder Copula
df_trans_werte_minusEins = df_trans_werte.copy()
df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] = df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] -1
df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

In [29]:
kunden_pos = [384, 446, 502, 599, 421, 285, 407, 499, 348, 351, 364, 449, 528, 414]

In [30]:
# Zeilen löschen
df_NO_minusEins = df_NO_minusEins.drop(kunden_pos).reset_index(drop = True)
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)
df_trans_werte = df_trans_werte.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)
df_NO_werte = df_NO_werte.drop(kunden_pos).reset_index(drop = True)
df_LY = df_LY.drop(kunden_pos).reset_index(drop = True)
df_trans_werte_minusEins = df_trans_werte_minusEins.drop(kunden_pos).reset_index(drop = True)

In [31]:
anzahl_kunden = len(df_HY["iid"])
anzahl_kunden

633

In [32]:
anzahl_monate = len(df_HY.columns[df_HY.columns != 'iid'])
anzahl_monate

15

In [33]:
anzahl_states = 2

## Ermittlung "typischer" Kunde

In [ ]:
# O-monat

In [49]:
O_monat_minusEins = df_NO_werte.copy()
for monat in range(1, anzahl_monate +1):
    O_monat_minusEins[str(monat)] = O_monat_minusEins[str(monat)].apply(lambda x: int(x > 0))

summe = O_monat_minusEins.loc[:, O_monat_minusEins.columns != 'iid'].sum().sum()
summe/(anzahl_monate * anzahl_kunden)

0.579989468141127

In [50]:
# y-monat

In [52]:
Y_monat_minusEins = df_trans_werte.copy()
for monat in range(1, anzahl_monate +1):
    Y_monat_minusEins[str(monat)] = Y_monat_minusEins[str(monat)].apply(lambda x: int(x > 0))

summe = Y_monat_minusEins.loc[:, Y_monat_minusEins.columns != 'iid'].sum().sum()
summe/(anzahl_monate * anzahl_kunden)

0.32090573986308585

### Herauslese der geschätzten Parameter

In [92]:
estimted_params_2 = [ 0.        , -1.84134701,  0.        ,  0.98963076,  0.        ,
        2.68214831,  0.        ,  2.80627766,  0.        , -1.2810976 ,
        0.        ,  0.69838679,  0.        ,  1.45855322,  0.        ,
       -2.13009073,  0.        , -0.04889301,  0.        ,  0.62177964,
        0.33148307,  1.39588074, -1.12147141,  7.70110619,  8.        ,
       -1.20879416,  1.99467476, -1.11065899, -1.89390642,  0.01515651,
        0.17867123, -0.03878318, -0.03539839,  1.03831034]

In [93]:
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4

alpha = estimted_params_2[:alpha_size]
gamma = estimted_params_2[alpha_size:alpha_size+gamma_size]
alpha_o = estimted_params_2[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
beta_o = estimted_params_2[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
delta_0 = estimted_params_2[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
delta_1 = estimted_params_2[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
alpha_p = estimted_params_2[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
beta_p_1 = estimted_params_2[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
beta_p_2 = estimted_params_2[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
theta = estimted_params_2[alpha_size+gamma_size+anzahl_states*6+1:]

In [36]:
alpha

[0.0, -2.57542179, 0.0, 2.62717551]

In [182]:
def Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat):
    
    state_t_minusEins = state_t_minusEins -1
    state = state -1
    str_monat = str(monat)
    
    # Definition der X_t_minusEins
    # verwendete Parameter, hier anpassen um verschiedene tests durchzuführen
    EM_monat_minusEins = EM_tryout[str_monat]
    Y_monat_minusEins = df_ones[str_monat]
    O_monat_minusEins = df_null[str_monat]
    # Definition
    X_t_minusEins = [O_monat_minusEins.apply(lambda x: int(x > 0)),\
                     Y_monat_minusEins.apply(lambda x: int(x > 0)),\
                     EM_monat_minusEins, np.square(EM_monat_minusEins)]
    
    X_t_minusEins = np.array(X_t_minusEins)
    array_v = alpha[anzahl_states*state_t_minusEins + state] +\
                    gamma[anzahl_states*state_t_minusEins + state] * X_t_minusEins[0] +\
                    gamma[(np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[1] +\
                    gamma[(2* np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[2] +\
                    gamma[(3* np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[3]

    return array_v

# Definition der Transitionmatrix
def probability_Q_state_monat(alpha, gamma):
    
    ### Erstellung leerer anzahl_states x anzahl_states arrays für jeden Kunden und jeden Monat
    Q = np.zeros((anzahl_kunden, anzahl_monate - 1, anzahl_states, anzahl_states))
                
    for monat in range(2, anzahl_monate + 1):
        
        for state_t_minusEins in range(1, anzahl_states +1):
            
            # Erste Spalte besteht aus Nullen, da eine Trasition zurück in state 1 nicht möglich ist
            for state in range(2, anzahl_states +1):
                
                q_array_iid_monat_state = Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat)

                # Speicherung der Werte in den leeren arrays
                for iid in range(0, anzahl_kunden):
                    
                    #Neue Definition ohne die Verwedung von Namen
                    Q[iid, monat-2, state_t_minusEins-1, state-1] = q_array_iid_monat_state[iid]
                    

    Q[:,:,:,0] = 0.0
    denominator = np.sum(np.exp(Q[..., 1:]), axis=3, keepdims=True)
    Q = np.exp(Q)/(1+denominator)
    
    return Q

### Q Berechnun für verschiedene Kunden
#### Treffe folgende Anpassungen, um Werte zu erhalten:
#### EM_monat_minusEins = EM_tryout[str_monat]
#### Y_monat_minusEins = df_ones[str_monat] , hier ones oder null zur Apassung ob im vorherigen monat ein Kauf getätit wurde
#### O_monat_minusEins = df_null[str_monat] , hier ones oder null zur Apassung ob im vorherigen monat eine Öffnung getätit wurde

In [75]:
# Erstelle ein DataFrame mit derselben Struktur und Anzahl an Zeilen wie df_trans_werte
df_null = pd.DataFrame(index=df_trans_werte.index, columns=df_trans_werte.columns)

# Fülle das DataFrame mit Nullen
df_null = df_null.fillna(0)

In [76]:
# Erstelle ein DataFrame mit derselben Struktur und Anzahl an Zeilen wie df_trans_werte
df_ones = pd.DataFrame(index=df_trans_werte.index, columns=df_trans_werte.columns)

# Fülle das DataFrame mit Nullen
df_ones = df_ones.fillna(1)

In [181]:
# Erstelle ein DataFrame mit derselben Struktur und Anzahl an Zeilen wie df_trans_werte
EM_tryout = pd.DataFrame(index=df_trans_werte.index, columns=df_trans_werte.columns)

# Fülle das DataFrame mit Nullen
EM_tryout = df_ones.fillna(1) * 3

### Nach oben beschriebenen Anpassungen, können un die Übergangswahrscheinlihkeiten dargestellt werden

In [183]:
probs = probability_Q_state_monat(alpha, gamma)

In [184]:
probs[10]

array([[[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]],

       [[0.30721371, 0.69278629],
        [0.29033407, 0.70966593]]])